In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras

Using TensorFlow backend.


In [3]:
from os import listdir
path = "E:\Datasets\KDEF_and_AKDEF\\KDEF"
pathlist = listdir(path)

In [19]:
from shutil import copyfile
import os

for i in range(len(pathlist)):
    img_path = listdir(path+"\\"+pathlist[i])
    for j in range(len(img_path)):
        if(img_path[j][4:6]=="AF"):
            dst = "E:\\Datasets\\KDEF_and_AKDEF\\data\\AF"
        elif(img_path[j][4:6]=="AN"):
            dst = "E:\\Datasets\\KDEF_and_AKDEF\\data\\AN"
        elif(img_path[j][4:6]=="DI"):
            dst = "E:\\Datasets\\KDEF_and_AKDEF\\data\\DI"
        elif(img_path[j][4:6]=="HA"):
            dst = "E:\\Datasets\\KDEF_and_AKDEF\\data\\HA"
        elif(img_path[j][4:6]=="NE"):
            dst = "E:\\Datasets\\KDEF_and_AKDEF\\data\\NE"
        elif(img_path[j][4:6]=="SA"):
            dst = "E:\\Datasets\\KDEF_and_AKDEF\\data\\SA"
        elif(img_path[j][4:6]=="SU"):
            dst = "E:\\Datasets\\KDEF_and_AKDEF\\data\\SU"
        
        if not os.path.exists(dst):
            os.makedirs(dst)
        
        src = path+"\\"+pathlist[i]+"\\"+img_path[j]
        dst = dst + "\\" + img_path[j]
        try:
            copyfile(src, dst)
        except:
            pass

# Training with CNN

In [20]:
from keras.datasets import mnist
from keras.utils import to_categorical

from keras import models
from keras import layers

from keras import losses, optimizers, metrics


In [21]:
from keras.preprocessing.image import ImageDataGenerator
train_dir = 'E:\\Datasets\\KDEF_and_AKDEF\\data'
test_dir ='E:\\Datasets\\KDEF_and_AKDEF\\data'


train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (150, 150),
    batch_size=32
)

validation_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size = (150, 150),
    batch_size=32
)

Found 4898 images belonging to 7 classes.
Found 4898 images belonging to 7 classes.


In [22]:
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPool2D((2, 2)))
model.add(layers.Conv2D(64, (3,3), activation='relu'))
model.add(layers.MaxPool2D((2,2)))
model.add(layers.Conv2D(64, (3,3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(7, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 64)        36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 73984)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               9470080   
__________

In [ ]:
model.compile(optimizer=optimizers.RMSprop(),
              loss=losses.categorical_crossentropy,
              metrics=['accuracy']
             )

history = model.fit_generator(
    train_generator,
    steps_per_epoch = 155,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=10
)

Epoch 1/30
155/155 [==============================] - 33s - loss: 2.0056 - acc: 0.2077 - val_loss: 1.7627 - val_acc: 0.2656
Epoch 2/30
155/155 [==============================] - 28s - loss: 1.3193 - acc: 0.4970 - val_loss: 1.0385 - val_acc: 0.6469
Epoch 3/30
155/155 [==============================] - 28s - loss: 0.8992 - acc: 0.6692 - val_loss: 0.6610 - val_acc: 0.7750
Epoch 4/30
155/155 [==============================] - 28s - loss: 0.6225 - acc: 0.7704 - val_loss: 0.3848 - val_acc: 0.8719
Epoch 5/30
154/155 [============================>.] - ETA: 0s - loss: 0.4026 - acc: 0.8523

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc)+1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()
plt.plot(epochs, loss, 'bo', label='Training losses')
plt.plot(epochs, val_loss, 'b', label='Validation losses')
plt.title('Training and validation loss')
plt.legend()

plt.show()

## Try using pretrined model

In [ ]:
from keras.applications import VGG16

conv_base = VGG16(weights='imagenet',
                 include_top=False,
                 input_shape=(150, 150, 3))

In [ ]:
model = models.Sequential()

model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(7, activation='softmax'))
model.summary()

In [ ]:
conv_base.trainable = False
print('This is the number of trainable weights before freezing the conv base:', len(model.trainable_weights))

model.compile(
    optimizer=optimizers.RMSprop(lr=1e-4), 
    loss=losses.categorical_crossentropy, 
    metrics=[metrics.categorical_accuracy]
)

history = model.fit_generator(train_generator, 
                              steps_per_epoch=155, 
                              epochs=15, 
                              validation_data=validation_generator, 
                              validation_steps=10)

In [ ]:
conv_base.trainable = True
set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [ ]:
model.compile(
    optimizer=optimizers.RMSprop(lr=1e-4), 
    loss=losses.categorical_crossentropy, 
    metrics=[metrics.categorical_accuracy]
)

history = model.fit_generator(train_generator, 
                              steps_per_epoch=155, 
                              epochs=30, 
                              validation_data=validation_generator, 
                              validation_steps=10)

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['categorical_accuracy']
val_acc = history.history['val_categorical_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc)+1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()
plt.plot(epochs, loss, 'bo', label='Training losses')
plt.plot(epochs, val_loss, 'b', label='Validation losses')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
model.save('finetuning_on_pretrained_vgg16.h5')